In [1]:
import os
import itertools
from spacy.en import English
import numpy as np

In [2]:
os.chdir("..")
os.getcwd()

'/mnt/Storage/Coding_Projects/Candidate_Classifier'

In [3]:
os.getcwd()

'/mnt/Storage/Coding_Projects/Candidate_Classifier'

In [4]:
from candidate_classifier.string_processing import *
from candidate_classifier.nltk_model.ngram_classifier import NgramClassifier

from candidate_classifier.tokenizers import *

KeyboardInterrupt: 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import make_scorer, classification_report, f1_score
from sklearn.grid_search import GridSearchCV
# from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.multiclass import OneVsOneClassifier

import codecs
from pprint import pprint
from time import time
import logging

In [7]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
fmt = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
hndlr = logging.StreamHandler()
hndlr.setFormatter(fmt)
logger.addHandler(hndlr)

In [8]:
sents_path = 'candidate_classifier/data/processed/clean_sents.txt'
labels_path = 'candidate_classifier/data/processed/sent_labels.txt'

In [10]:
with codecs.open(sents_path, mode='r', encoding='utf-8') as _f:
    docs = [l.strip() for l in _f]

with codecs.open(labels_path, mode='r', encoding='utf-8') as _f:
    labels = [l.strip() for l in _f]

candidates = sorted(list(set(labels)))
print candidates

[u'BUSH', u'CARSON', u'CHRISTIE', u'CLINTON', u'CRUZ', u'KASICH', u'RUBIO', u'SANDERS', u'TRUMP']


In [11]:
toknizrs = (simple_tokenizer,
            lemmas_tokenizer, 
            lemmas_no_punct,
            lemmas_merge_np,
            lemmas_merge_ents,
            lemmas_merge_np_merge_ents,
            lemmas_cased_tokenizer,
            lemmas_cased_merge_ents)

ng_tokenizers1 = [TransformerABC(prefilter_substitutions=['strip'], tokenizer=t) for t in toknizrs]

ng_tokenizers2 = [TransformerABC(prefilter_substitutions=['strip', 'lower'], tokenizer=t) for t in toknizrs]

In [14]:
ng_data1 = [list(t(docs)) for t in ng_tokenizers1]

In [13]:
ng_data2 = [list(t(docs)) for t in ng_tokenizers2]

In [14]:
ngram_pipe = Pipeline([
        ('clf', OneVsOneClassifier(NgramClassifier(pad_ngrams=True)))
])

In [14]:
for i, d in enumerate(ng_data2):
    print toknizrs[i].func_name
    scores = cross_val_score(ngram_pipe, np.asarray(d), y=labels, cv=3, scoring='f1_weighted')
    print scores
    print np.mean(scores)
    print '\n'

simple_tokenizer
[ 0.54421309  0.53359132  0.53997209]
0.53925883617


lemmas_tokenizer
[ 0.57717623  0.5633713   0.58675491]
0.575767479881


lemmas_no_punct
[ 0.57213578  0.55784761  0.58247465]
0.570819346765


lemmas_merge_np
[ 0.56219524  0.54191144  0.55888021]
0.554328964724


lemmas_merge_ents
[ 0.57936097  0.5676273   0.58711447]
0.578034248422


lemmas_merge_np_merge_ents
[ 0.56193059  0.54320551  0.56146713]
0.55553440775


lemmas_cased_tokenizer
[ 0.57684843  0.5633713   0.5867876 ]
0.575669107958


lemmas_cased_merge_ents
[ 0.57936097  0.5676273   0.58711447]
0.578034248422




In [14]:
# Ratio calculated in normal space
scores = cross_val_score(ngram_pipe, np.asarray(ng_data2[4]), y=labels, cv=3, scoring='f1_weighted')
print scores
print np.mean(scores)

[ 0.57936097  0.5676273   0.58711447]
0.578034248422


In [13]:
# Ratio in normal space and predict_proba in normal space
scores = cross_val_score(ngram_pipe, np.asarray(ng_data2[4]), y=labels, cv=3, scoring='f1_weighted')
print scores
print np.mean(scores)

[ 0.57936097  0.5676273   0.58711447]
0.578034248422


In [16]:
# Ratio in normal space and predict_proba in normal space
# data not lowercased
scores = cross_val_score(ngram_pipe, np.asarray(ng_data1[4]), y=labels, cv=3, scoring='f1_weighted')
print scores
print np.mean(scores)

[ 0.57520839  0.56867061  0.58508877]
0.576322588505


In [13]:
# Git version: aff3e929ec62bae7041130e447bfbc9b07f5567f
scores = cross_val_score(ngram_pipe, np.asarray(ng_data2[4]), y=labels, cv=3, scoring='f1_weighted')
print scores
print np.mean(scores)

[ 0.57936097  0.5676273   0.58711447]
0.578034248422


In [15]:
# Git version: aff3e929ec62bae7041130e447bfbc9b07f5567f
# No lowercase
scores = cross_val_score(ngram_pipe, np.asarray(ng_data1[4]), y=labels, cv=3, scoring='f1_weighted')
print scores
print np.mean(scores)

[ 0.57520839  0.56867061  0.58508877]
0.576322588505


In [15]:
# Git version: aff3e929ec62bae7041130e447bfbc9b07f5567f
# Scipy 17.1
scores = cross_val_score(ngram_pipe, np.asarray(ng_data2[4]), y=labels, cv=3, scoring='f1_weighted')
print scores
print np.mean(scores)

[ 0.57936097  0.5676273   0.58711447]
0.578034248422
